## Convert the R1CS into a QAP


## Over Real Number

In [11]:
import numpy as np
import random

# Define the matrices
A = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

B = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

C = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick random values for x and y
x = random.randint(1,1000)
y = random.randint(1,1000)

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])

print("witness {}".format(w))

result = C.dot(w) == np.multiply(A.dot(w),B.dot(w))
assert result.all(), "result contains an inequality"

witness [         1 1500761992        959        543    2759043 1498160349]


In [27]:
from scipy.interpolate import lagrange
import numpy as np
from numpy.polynomial import polynomial as P

np.set_printoptions(precision=2)


def poly_interpolate(index, matrics):
    res = []
    for row in matrics:
        poly = lagrange(index, row).coef[::-1]
        res.append(poly)
    return np.array(res, dtype=object)


def polymuln(lst):
    if len(lst) == 0:
        return 1
    return P.polymul(lst[0], polymuln(lst[1:]))


def QAP(s, A, B, C):
    L = np.matmul(s, A)
    R = np.matmul(s, B)
    O = np.matmul(s, C)
    print(f"L: {L}")
    print(f"R: {R}")
    print(f"O: {O}")
    Q = P.polysub(P.polymul(L, R), O)
    print(f"T:\n {Q}")
    assert len(L) == len(R) == len(O)
    return P.polydiv(Q, Z(len(L)))


def Z(n):
    ns = [[-(1 + i), 1] for i, _ in enumerate(range(n))]
    return polymuln(ns)


def main(s, A, B, C):
    x = np.arange(1, len(A) + 1)

    resA = poly_interpolate(x, np.transpose(A))
    resB = poly_interpolate(x, np.transpose(B))
    resC = poly_interpolate(x, np.transpose(C))

    print(f"interporated A: \n {resA}")
    print(f"interporated B: \n {resB}")
    print(f"interporated C: \n {resC}")

    return QAP(s, resA, resB, resC)





print(main(w, A, B, C))

interporated A: 
 [array([0.]) array([0.]) array([10., -9.,  2.]) array([0.])
 array([-3.,  4., -1.]) array([0.])]
interporated B: 
 [array([0.]) array([0.]) array([ 3. , -2.5,  0.5])
 array([ 2. , -3.5,  1.5]) array([0.]) array([0.])]
interporated C: 
 [[-3.0 4.5 -1.5]
 [1.0 -1.5 0.5]
 [1.0 -1.5 0.5]
 [2.0 -3.0 1.0]
 [3.0 -2.5 0.5]
 [-4.0 5.5 -1.5]]
L: [-8267539. 11027541. -2757125.]
R: [ 3963. -4298.  1294.]
O: [-4483600233.0 5981838261.0 -1495478985.0]
T:
 [-28280656824.0 73254189344.0 -67525574074.0 26119761304.0 -3567719750.0]
(array([4713442804.0, -3567719750.0], dtype=object), array([0.0], dtype=object))


## Over Finite Field

In [40]:
import numpy as np
import random
import galois
GF = galois.GF(79)

# Define the matrices
A_F = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

B_F = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

C_F = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [GF(79-3),1,1,2,0,GF(79-1)]])

A_galois = GF(A_F)
B_galois = GF(B_F)
C_galois = GF(C_F)

# pick random values for x and y
x = GF(1)
y = GF(2)


# this is our original formula
v1 = 3*x*x
v2 = v1 * y
out = 3*x*x*y + 5*x*y + GF(79-1)*x + GF(79-2)*y + GF(3)
# the witness vector with the intermediate variables inside
w = GF(np.array([1, out, x, y, v1, v2]))

print("witness {}".format(w))

result = C_galois.dot(w) == np.multiply(A_galois.dot(w),B_galois.dot(w))
assert result.all(), "result contains an inequality"

witness [ 1 14  1  2  3  6]


In [114]:
from scipy.interpolate import lagrange
import numpy as np
import functools 
from numpy.polynomial import polynomial as P

import galois
GF = galois.GF(79)

np.set_printoptions(precision=2)


def poly_finite_field_interpolate(index, matrics):
    res = []
    for row in matrics:
        poly = galois.lagrange_poly(index, row)
        coef = poly.coefficients()[::-1]
        res.append(coef)
    return np.array(res, dtype=object)


def polymuln(lst):
    if len(lst) == 0:
        return 1
    return P.polymul(lst[0], polymuln(lst[1:]))

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return functools.reduce(sum_, map(mul_, polys, witness))


def QAP(s, A, B, C):

    L = inner_product_polynomials_with_witness(s, A)
    R = inner_product_polynomials_with_witness(s, B)

    C_int = np.array(C, dtype=int)    
    O = inner_product_polynomials_with_witness(s, GF(C_int))

    print(f"L: {L}")
    print(f"R: {R}")
    print(f"O: {O}")

    print(f"L: {galois.Poly(L[::-1])}")
    print(f"R: {galois.Poly(R[::-1])}")
    print(f"O: {galois.Poly(O[::-1])}")

    

    Q = P.polysub(P.polymul(L, R), O)
    print(f"h(x)t(x):\n {Q}")

    H = L * R - O
    print(f"H: {H}")

    T = galois.Poly([1, 79-1], field=GF)
    for i in range(2, L.shape[0] + 1):
        T *= galois.Poly([1, 79-i], field=GF)

    print("\nT = ", T)

    H = (galois.Poly(L[::-1]) * galois.Poly(R[::-1]) - galois.Poly(O[::-1])) // T
    print("\nH = ",H)
    rem = (galois.Poly(L[::-1]) * galois.Poly(R[::-1]) - galois.Poly(O[::-1])) % T
    print("\nrem = ", rem)

    assert rem == 0


    # assert L * R == O + H * T, "division has a remainder"

    for i in range(1, L.shape[0] + 2):
        print(f"T({i}) = ", T(i))
        if i == L.shape[0]:
            print("-"*10)

    tau = GF(20)
    T_tau = T(tau)
    print(f"\nT(τ) = {T_tau}")

    assert len(L) == len(R) == len(O)
    return P.polydiv(Q, Z(len(L)))


def Z(n):
    ns = [[-(1 + i), 1] for i, _ in enumerate(range(n))]
    return polymuln(ns)


def main(s, A, B, C):
    x = GF(np.arange(1, len(A) + 1))

    A_galois = GF(A)
    B_galois = GF(B)
    C_galois = GF(C)

    resA = poly_finite_field_interpolate(x, np.transpose(A_galois))
    resB = poly_finite_field_interpolate(x, np.transpose(B_galois))
    resC = poly_finite_field_interpolate(x, np.transpose(C_galois))

    print(f"interporated A: \n {resA}")
    print(f"interporated B: \n {resB}")
    print(f"interporated C: \n {resC}")

    return QAP(s, resA, resB, resC)




print(f"h(x): \n {main(w, A_F, B_F, C_F)}")

interporated A: 
 [GF([0], order=79) GF([0], order=79) GF([10, 70,  2], order=79)
 GF([0], order=79) GF([76,  4, 78], order=79) GF([0], order=79)]
interporated B: 
 [GF([0], order=79) GF([0], order=79) GF([ 3, 37, 40], order=79)
 GF([ 2, 36, 41], order=79) GF([0], order=79) GF([0], order=79)]
interporated C: 
 [[76 44 38]
 [1 38 40]
 [1 38 40]
 [2 76 1]
 [3 37 40]
 [75 45 38]]
L: [ 1  3 78]
R: [ 7 30 43]
O: [ 1 41 40]
L: 78x^2 + 3x + 1
R: 43x^2 + 30x + 7
O: 40x^2 + 41x + 1
h(x)t(x):
 [   6.   10.  639. 2469. 3354.]
H: [ 6 49 75]

T =  x^3 + 73x^2 + 11x + 73

H =  36x + 78

rem =  0
T(1) =  0
T(2) =  0
T(3) =  0
----------
T(4) =  6

T(τ) = 47
h(x): 
 (array([22593.,  3354.]), array([ 135564., -228389.,   99303.]))


In [111]:
mtxs = [A_F, B_F, C_F]
poly_m = []

for m in mtxs:
    poly_list = []
    for i in range(0, m.shape[1]):
        points_x = GF(np.zeros(m.shape[0], dtype=int))
        points_y = GF(np.zeros(m.shape[0], dtype=int))
        for j in range(0, m.shape[0]):
            points_x[j] = GF(j+1)
            points_y[j] = m[j][i]

        poly = galois.lagrange_poly(points_x, points_y)
        coef = poly.coefficients()[::-1]
        if len(coef) < m.shape[0]:
            coef = np.append(coef, np.zeros(m.shape[0] - len(coef), dtype=int))
        poly_list.append(coef)
    
    poly_m.append(GF(poly_list))

Lp = poly_m[0]
Rp = poly_m[1]
Op = poly_m[2]

U = galois.Poly((w @ Lp)[::-1])
V = galois.Poly((w @ Rp)[::-1])
W = galois.Poly((w @ Op)[::-1])

print("U = ", U)
print("V = ", V)
print("W = ", W)


U =  78x^2 + 3x + 1
V =  43x^2 + 30x + 7
W =  40x^2 + 41x + 1


In [90]:
import galois
GF = galois.GF(79)

y_p = GF(np.array([3,0,1]))

xs = GF(np.array([1,2,3]))
xs_1 = galois.lagrange_poly(xs, y_p)
coef = xs_1.coefficients()[::-1]

print(np.array([3,0,1]))
print(GF(np.array([3,0,1])))

print(xs_1)
# print(xs_1[:2])
print(coef)


y_p2 = np.array([3,0,1])
xs_ = np.array([1,2,3])
xs_1_ = lagrange(xs_, y_p2).coef[::-1]
xs_1__ = lagrange(xs_, y_p2)

print(xs_1_)
print(xs_1__)



ns = [[-(1 + i), 1] for i, _ in enumerate(range(4))]
print(ns)

ns2 = [[-(1 + i), 1] for i, _ in enumerate(range(4))]
print(galois.Poly([1, 78], field = GF))
galois.Poly(ns, field=GF)



[3 0 1]
[3 0 1]
2x^2 + 70x + 10
[10 70  2]
[10. -9.  2.]
   2
2 x - 9 x + 10
[[-1, 1], [-2, 1], [-3, 1], [-4, 1]]
x + 78


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()